In [15]:
import numpy as np
import math
import copy
import random as rd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math


In [16]:
rng = np.random.RandomState(54321)

x_data = rng.uniform(-10,10,200) + rng.randn(200)
y_data = x_data*x_data

x_data=x_data[:, np.newaxis]
y_data=y_data[:, np.newaxis]

In [37]:
def create_mini_batches(X, y, batch_size): 
    mini_batches_x = [] 
    mini_batches_y = []
    data = np.hstack((X, y)) 
    np.random.shuffle(data) 
    n_minibatches = data.shape[0] // batch_size 
    i = 0

    for i in range(n_minibatches + 1): 
        mini_batch = data[i * batch_size:(i + 1)*batch_size, :] 
        X_mini = mini_batch[:, :-1] 
        Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        mini_batches_x.append(X_mini) 
        mini_batches_y.append(Y_mini)
    if data.shape[0] % batch_size != 0: 
        mini_batch = data[i * batch_size:data.shape[0]] 
        X_mini = mini_batch[:, :-1] 
        Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        mini_batches_x.append(X_mini) 
        mini_batches_y.append(Y_mini) 
    return mini_batches_x, mini_batches_y 

In [42]:
hl_dict = {0: 4, 1: 7}

In [140]:
red = Network(x_data,y_data,hl_dict,10)
red.train()

[array([1770.07167796]),
 array([974.59240243]),
 array([2591.07350402]),
 array([1974.73689299]),
 array([2900.55636163]),
 array([4345.7866954]),
 array([1325.04232775]),
 array([3044.80622251]),
 array([1634.00343881]),
 array([2673.42125306]),
 array([2588.42395601]),
 array([3073.92583759]),
 array([2919.66825583]),
 array([1410.22237891]),
 array([1436.63338154]),
 array([3576.44221108]),
 array([1770.8461723]),
 array([1229.86765987]),
 array([1223.23590881]),
 array([1458.71097358])]

In [139]:
class Network:
    def __init__(self,x_data,y_data,hl_dict,bz):
        self.x_data = x_data #initial layer
        self.values = y_data #valores reales
        self.input_neurons = len(x_data[0])
        self.hl_dict = hl_dict # {0: 20, 1: 33 ...}
        self.layers = []
        self.output = 0
        self.output_layer = None
        self.batch_size = bz
        self.create_layers()
        
    def create_layers(self):
        n = len(self.hl_dict.keys())
        for j in range(0,n):
            if j==0:
                input_size = len(self.x_data[0])
            else:
                input_size = self.hl_dict[j-1]
            self.layers.append(Layer(self.hl_dict[j],input_size))
        
        input_size = self.hl_dict[n-1]
        self.output_layer = Layer(1,input_size)
        
    def train(self):
        costs = []
        mbx, mby = self.create_mini_batches(self.x_data,self.values,self.batch_size)
        #print(len(mbx))
        for minix,miniy in zip(mbx,mby):
            batch_cost = self.feed_forward(minix,miniy)
            costs.append(batch_cost)
            #self.back_propagation(batch_cost)
        return costs
            
    def create_mini_batches(self,X, y, batch_size): 
        mini_batches_x = [] 
        mini_batches_y = []
        data = np.hstack((X, y)) 
        np.random.shuffle(data) 
        n_minibatches = data.shape[0] // batch_size 
        i = 0

        for i in range(n_minibatches): 
            mini_batch = data[i * batch_size:(i + 1)*batch_size, :] 
            X_mini = mini_batch[:, :-1] 
            Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
            mini_batches_x.append(X_mini) 
            mini_batches_y.append(Y_mini)
        if data.shape[0] % batch_size != 0: 
            mini_batch = data[i * batch_size:data.shape[0]] 
            X_mini = mini_batch[:, :-1] 
            Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
            mini_batches_x.append(X_mini) 
            mini_batches_y.append(Y_mini) 
        return mini_batches_x, mini_batches_y 
            
        
    def feed_forward(self,minibatch_x,ys): #por mini batches para hacer backprop
        i = 0
        acc_cost = 0
        n = len(minibatch_x)
        #print(n)
        for observation in minibatch_x:
            cost = self.forward(observation,ys[i])
            i = i+1
            acc_cost += cost
        
        average_cost = acc_cost/n
        return average_cost
        
    def forward(self,x_obsv,y_real):
        out = x_obsv
        for layer in self.layers:
            #print(layer)
            layer.inputs = out
            #print(layer.inputs)
            layer.forward()
            out = layer.outputs
            
        ol = self.output_layer
        ol.inputs = out
        #print(ol.inputs)
        ol.forward()
        y_predicted = ol.outputs
        
        Cost = self.MSE(y_predicted,y_real)
        return Cost
            
    def MSE(self,y_pred,y_real):
        return (y_real-y_pred)**2
        
    
class Layer:
    def __init__(self,cl,pl): #len(inputs) = pl por lo tanto len(outputs) = nl
        self.weights = np.random.rand(cl,pl)
        self.inputs = []
        self.bias = np.random.rand(cl,1)
        self.neurons = self.define_neurons(cl)
        self.outputs = [0]*cl
        self.neuron_count = cl
        
    def define_neurons(self,n):
        neuronas = []
        for i in range(0,n):
            ins = self.inputs
            weights = self.weights[i]
            bias = self.bias[i]
            new_neuron = Neuron(ins,weights,bias)
            neuronas.append(new_neuron)
        return neuronas
            
    def forward(self):
       #print('inputs de la capa: '+str(self.inputs))
        for i in range(0,self.neuron_count):
            current_neuron = self.neurons[i]
            current_neuron.inputs = self.inputs
            
            self.outputs[i] = current_neuron.calculate_a()
      
    
class Neuron:
    def __init__(self,iv,vw,b):
        self.wprevious = vw #input weight vector
        self.bias = b #input bias
        self.inputs = iv

        
    def calculate_a(self):
        net = self.calculate_net()
        a = self.sigmoid(net) #¿tengo que meterle relu?
        #print('a :'+str(a))
        return a
    
    def calculate_net(self):
        #print('wprevious: '+str(self.wprevious))
        #print('ins: '+str(self.inputs))
        hadamard = np.multiply(self.wprevious,self.inputs)
        #print('bias: '+str(self.bias))
        suma = sum(hadamard) + self.bias[0]
        #print('suma: '+str(suma))
        return suma
        
    def relu(self, x):
        return x * (x > 0)

    def sigmoid(self, x):
        return 1/(1-np.exp(-x))

In [11]:
a = np.zeros((2,3))

In [12]:
a[1]

array([0., 0., 0.])